In [3]:
# загрузим библиотеки и установим опции

from __future__ import division, print_function
# отключим всякие предупреждения Anaconda
import warnings
warnings.filterwarnings('ignore')
#%matplotlib inline

from matplotlib import pyplot as plt
plt.style.use('ggplot')
%matplotlib
import seaborn as sns


import pickle
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from scipy.sparse import hstack
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression

import re

Using matplotlib backend: Qt5Agg


In [2]:
import xgboost as xgb

/home/dz/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [3]:
import keras

Using Theano backend.


In [5]:
test_df = pd.read_csv('data/test_sessions.csv')
times = ['time%s' % i for i in range(1, 11)]
test_df[times] = test_df[times].apply(pd.to_datetime)
test_df = test_df.sort_values(by='time1')
test_df

,session_id,site1,time1,site2,time2,site3,time3,site4,time4,site5,...,site6,time6,site7,time7,site8,time8,site9,time9,site10,time10
65539,65540,21,2014-05-01 17:14:03,NaN,NaT,NaN,NaT,NaN,NaT,NaN,...,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT
64198,64199,23,2014-05-02 07:52:08,66.0,2014-05-02 07:54:08,63.0,2014-05-02 07:54:08,2626.0,2014-05-02 07:55:09,NaN,...,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT
2267,2268,979,2014-05-02 07:57:51,73.0,2014-05-02 07:59:34,NaN,NaT,NaN,NaT,NaN,...,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT
29733,29734,66,2014-05-02 08:05:16,69.0,2014-05-02 08:05:17,67.0,2014-05-02 08:05:17,70.0,2014-05-02 08:05:17,71.0,...,68.0,2014-05-02 08:05:17,71.0,2014-05-02 08:05:18,70.0,2014-05-02 08:05:18,69.0,2014-05-02 08:05:18,67.0,2014-05-02 08:05:18
77047,77048,167,2014-05-02 08:05:32,167.0,2014-05-02 08:05:33,359.0,2014-05-02 08:05:34,167.0,2014-05-02 08:05:34,167.0,...,305.0,2014-05-02 08:09:19,306.0,2014-05-02 08:09:20,306.0,2014-05-02 08:09:22,979.0,2014-05-02 08:09:54,68.0,2014-05-02 08:12:46
33525,33526,67,2014-05-02 08:06:03,70.0,2014-05-02 08:06:03,69.0,2014-05-02 08:06:03,68.0,2014-05-02 08:06:03,71.0,...,66.0,2014-05-02 08:06:03,67.0,2014-05-02 08:06:04,69.0,2014-05-02 08:06:04,69.0,2014-05-02 08:06:06,70.0,2014-05-02 08:06:07
61452,61453,69,2014-05-02 08:06:07,167.0,2014-05-02 08:06:08,67.0,2014-05-02 08:06:09,71.0,2014-05-02 08:06:09,70.0,...,23.0,2014-05-02 08:06:25,21.0,2014-05-02 08:06:25,21.0,2014-05-02 08:06:40,22.0,2014-05-02 08:06:42,23.0,2014-05-02 08:06:43
17227,17228,23,2014-05-02 08:06:40,66.0,2014-05-02 08:08:40,63.0,2014-05-02 08:08:40,21.0,2014-05-02 08:09:30,21.0,...,22.0,2014-05-02 08:09:32,23.0,2014-05-02 08:09:32,797.0,2014-05-02 08:09:37,22.0,2014-05-02 08:09:41,23.0,2014-05-02 08:09:41
44867,44868,69,2014-05-02 08:06:46,71.0,2014-05-02 08:06:46,68.0,2014-05-02 08:06:46,66.0,2014-05-02 08:06:46,70.0,...,67.0,2014-05-02 08:06:46,67.0,2014-05-02 08:06:47,69.0,2014-05-02 08:06:47,70.0,2014-05-02 08:06:47,71.0,2014-05-02 08:06:48
81683,81684,167,2014-05-02 08:06:48,70.0,2014-05-02 08:06:48,21.0,2014-05-02 08:07:34,21.0,2014-05-02 08:07:48,23.0,...,23.0,2014-05-02 08:08:04,21.0,2014-05-02 08:08:04,63.0,2014-05-02 08:08:05,797.0,2014-05-02 08:08:08,973.0,2014-05-02 08:10:37


In [36]:
# загрузим обучающую и тестовую выборки
train_df = pd.read_csv('data/train_sessions.csv')#,index_col='session_id')
test_df = pd.read_csv('data/test_sessions.csv')#, index_col='session_id')

# приведем колонки time1, ..., time10 к временному формату
times = ['time%s' % i for i in range(1, 11)]
train_df[times] = train_df[times].apply(pd.to_datetime)
test_df[times] = test_df[times].apply(pd.to_datetime)

# отсортируем данные по времени
train_df = train_df.sort_values(by='time1')

# посмотрим на заголовок обучающей выборки
train_df.head()

,session_id,site1,time1,site2,time2,site3,time3,site4,time4,site5,...,time6,site7,time7,site8,time8,site9,time9,site10,time10,target
21668,21669,56,2013-01-12 08:05:57,55.0,2013-01-12 08:05:57,NaN,NaT,NaN,NaT,NaN,...,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT,0
54842,54843,56,2013-01-12 08:37:23,55.0,2013-01-12 08:37:23,56.0,2013-01-12 09:07:07,55.0,2013-01-12 09:07:09,NaN,...,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT,0
77291,77292,946,2013-01-12 08:50:13,946.0,2013-01-12 08:50:14,951.0,2013-01-12 08:50:15,946.0,2013-01-12 08:50:15,946.0,...,2013-01-12 08:50:16,948.0,2013-01-12 08:50:16,784.0,2013-01-12 08:50:16,949.0,2013-01-12 08:50:17,946.0,2013-01-12 08:50:17,0
114020,114021,945,2013-01-12 08:50:17,948.0,2013-01-12 08:50:17,949.0,2013-01-12 08:50:18,948.0,2013-01-12 08:50:18,945.0,...,2013-01-12 08:50:18,947.0,2013-01-12 08:50:19,945.0,2013-01-12 08:50:19,946.0,2013-01-12 08:50:19,946.0,2013-01-12 08:50:20,0
146669,146670,947,2013-01-12 08:50:20,950.0,2013-01-12 08:50:20,948.0,2013-01-12 08:50:20,947.0,2013-01-12 08:50:21,950.0,...,2013-01-12 08:50:21,946.0,2013-01-12 08:50:21,951.0,2013-01-12 08:50:22,946.0,2013-01-12 08:50:22,947.0,2013-01-12 08:50:22,0


In [38]:
train_df

,session_id,site1,time1,site2,time2,site3,time3,site4,time4,site5,...,time6,site7,time7,site8,time8,site9,time9,site10,time10,target
21668,21669,56,2013-01-12 08:05:57,55.0,2013-01-12 08:05:57,0.0,1970-01-01 00:00:00,0.0,1970-01-01 00:00:00,0.0,...,1970-01-01 00:00:00,0.0,1970-01-01 00:00:00,0.0,1970-01-01 00:00:00,0.0,1970-01-01 00:00:00,0.0,1970-01-01 00:00:00,0
54842,54843,56,2013-01-12 08:37:23,55.0,2013-01-12 08:37:23,56.0,2013-01-12 09:07:07,55.0,2013-01-12 09:07:09,0.0,...,1970-01-01 00:00:00,0.0,1970-01-01 00:00:00,0.0,1970-01-01 00:00:00,0.0,1970-01-01 00:00:00,0.0,1970-01-01 00:00:00,0
77291,77292,946,2013-01-12 08:50:13,946.0,2013-01-12 08:50:14,951.0,2013-01-12 08:50:15,946.0,2013-01-12 08:50:15,946.0,...,2013-01-12 08:50:16,948.0,2013-01-12 08:50:16,784.0,2013-01-12 08:50:16,949.0,2013-01-12 08:50:17,946.0,2013-01-12 08:50:17,0
114020,114021,945,2013-01-12 08:50:17,948.0,2013-01-12 08:50:17,949.0,2013-01-12 08:50:18,948.0,2013-01-12 08:50:18,945.0,...,2013-01-12 08:50:18,947.0,2013-01-12 08:50:19,945.0,2013-01-12 08:50:19,946.0,2013-01-12 08:50:19,946.0,2013-01-12 08:50:20,0
146669,146670,947,2013-01-12 08:50:20,950.0,2013-01-12 08:50:20,948.0,2013-01-12 08:50:20,947.0,2013-01-12 08:50:21,950.0,...,2013-01-12 08:50:21,946.0,2013-01-12 08:50:21,951.0,2013-01-12 08:50:22,946.0,2013-01-12 08:50:22,947.0,2013-01-12 08:50:22,0
242170,242171,952,2013-01-12 08:50:22,947.0,2013-01-12 08:50:23,953.0,2013-01-12 08:50:23,946.0,2013-01-12 08:50:23,947.0,...,2013-01-12 08:50:24,953.0,2013-01-12 08:50:24,955.0,2013-01-12 08:50:24,946.0,2013-01-12 08:50:25,947.0,2013-01-12 08:50:25,0
57156,57157,953,2013-01-12 08:50:25,947.0,2013-01-12 08:50:26,946.0,2013-01-12 08:50:26,953.0,2013-01-12 08:50:26,955.0,...,2013-01-12 08:50:27,953.0,2013-01-12 08:50:27,946.0,2013-01-12 08:50:27,953.0,2013-01-12 08:50:28,1033.0,2013-01-12 08:50:28,0
240200,240201,946,2013-01-12 08:50:28,947.0,2013-01-12 08:50:28,954.0,2013-01-12 08:50:28,953.0,2013-01-12 08:50:29,946.0,...,2013-01-12 08:50:29,946.0,2013-01-12 08:50:30,956.0,2013-01-12 08:50:30,957.0,2013-01-12 08:50:31,956.0,2013-01-12 08:50:31,0
210685,210686,946,2013-01-12 08:50:31,956.0,2013-01-12 08:50:32,946.0,2013-01-12 08:50:32,946.0,2013-01-12 08:50:33,955.0,...,2013-01-12 08:50:33,946.0,2013-01-12 08:50:34,946.0,2013-01-12 08:50:35,946.0,2013-01-12 08:50:36,948.0,2013-01-12 08:50:36,0
98803,98804,948,2013-01-12 08:50:37,946.0,2013-01-12 08:50:37,948.0,2013-01-12 08:50:38,784.0,2013-01-12 08:50:49,49.0,...,2013-01-12 08:51:03,812.0,2013-01-12 08:51:03,982.0,2013-01-12 08:51:03,52.0,2013-01-12 08:51:03,52.0,2013-01-12 08:51:04,0


In [37]:
train_df = train_df.fillna(0)

In [41]:
c = train_df['time3'][0]

In [42]:
c = train_df['time3'][0]
def isnan(x):
    if x==c:
        return 1
    else:
        return 0
is_nan = train_df['time1'].apply(isnan)
for i in range(2,11):
    is_nan = is_nan + train_df['time'+str(i)].apply(isnan )

In [44]:
is_nan

21668     8
54842     6
77291     0
114020    0
146669    0
242170    0
57156     0
240200    0
210685    0
98803     0
113493    0
223836    0
145474    0
186632    0
45192     0
102929    0
15788     0
205867    6
21101     0
193453    0
252923    0
12712     0
182015    1
55749     8
237108    0
120125    0
140104    0
129356    0
136698    0
113188    0
         ..
131401    0
72348     0
181939    0
67430     7
133255    0
78515     0
159244    0
6102      0
126231    0
62542     0
122172    0
182722    0
216795    0
88872     0
201959    0
99917     0
35922     0
91325     0
182877    0
36911     0
101405    0
66476     2
221865    8
101394    9
33801     8
12223     0
164437    0
12220     0
156967    0
204761    3
dtype: int64

In [5]:
sites = ['site%s' % i for i in range(1, 11)]
train_df[sites] = train_df[sites].fillna(0).astype('int').astype('str')
test_df[sites] = test_df[sites].fillna(0).astype('int').astype('str')

In [6]:
from gensim.models import word2vec

In [7]:
train_df['list'] = train_df['site1']
test_df['list'] = test_df['site1']
for s in sites[1:]:
    train_df['list'] = train_df['list']+","+train_df[s]
    test_df['list'] = test_df['list']+","+test_df[s]

In [8]:
train_df.head()

,session_id,site1,time1,site2,time2,site3,time3,site4,time4,site5,...,site7,time7,site8,time8,site9,time9,site10,time10,target,list
21668,21669,56,2013-01-12 08:05:57,55,2013-01-12 08:05:57,0,NaT,0,NaT,0,...,0,NaT,0,NaT,0,NaT,0,NaT,0,"56,55,0,0,0,0,0,0,0,0"
54842,54843,56,2013-01-12 08:37:23,55,2013-01-12 08:37:23,56,2013-01-12 09:07:07,55,2013-01-12 09:07:09,0,...,0,NaT,0,NaT,0,NaT,0,NaT,0,"56,55,56,55,0,0,0,0,0,0"
77291,77292,946,2013-01-12 08:50:13,946,2013-01-12 08:50:14,951,2013-01-12 08:50:15,946,2013-01-12 08:50:15,946,...,948,2013-01-12 08:50:16,784,2013-01-12 08:50:16,949,2013-01-12 08:50:17,946,2013-01-12 08:50:17,0,"946,946,951,946,946,945,948,784,949,946"
114020,114021,945,2013-01-12 08:50:17,948,2013-01-12 08:50:17,949,2013-01-12 08:50:18,948,2013-01-12 08:50:18,945,...,947,2013-01-12 08:50:19,945,2013-01-12 08:50:19,946,2013-01-12 08:50:19,946,2013-01-12 08:50:20,0,"945,948,949,948,945,946,947,945,946,946"
146669,146670,947,2013-01-12 08:50:20,950,2013-01-12 08:50:20,948,2013-01-12 08:50:20,947,2013-01-12 08:50:21,950,...,946,2013-01-12 08:50:21,951,2013-01-12 08:50:22,946,2013-01-12 08:50:22,947,2013-01-12 08:50:22,0,"947,950,948,947,950,952,946,951,946,947"


In [9]:
train_df['list_w'] = train_df['list'].apply(lambda x: x.split(','))
test_df['list_w'] = test_df['list'].apply(lambda x: x.split(','))

In [10]:
train_df.head(2)

,session_id,site1,time1,site2,time2,site3,time3,site4,time4,site5,...,time7,site8,time8,site9,time9,site10,time10,target,list,list_w
21668,21669,56,2013-01-12 08:05:57,55,2013-01-12 08:05:57,0,NaT,0,NaT,0,...,NaT,0,NaT,0,NaT,0,NaT,0,"56,55,0,0,0,0,0,0,0,0","[56, 55, 0, 0, 0, 0, 0, 0, 0, 0]"
54842,54843,56,2013-01-12 08:37:23,55,2013-01-12 08:37:23,56,2013-01-12 09:07:07,55,2013-01-12 09:07:09,0,...,NaT,0,NaT,0,NaT,0,NaT,0,"56,55,56,55,0,0,0,0,0,0","[56, 55, 56, 55, 0, 0, 0, 0, 0, 0]"


In [ ]:
def get_auc_lr_valid(X, y, C=1, seed=7, ratio = 0.8):
    # разделим выборку на обучающую и валидационную
    idx = round(X.shape[0] * ratio)
    # обучение классификатора
    lr = LogisticRegression(C=C, random_state=seed, n_jobs=-1).fit(X[:idx, :], y[:idx])
    # прогноз для валидационной выборки
    y_pred = lr.predict_proba(X[idx:, :])[:, 1]
    # считаем качество
    score = roc_auc_score(y[idx:], y_pred)
    
    return score

In [ ]:
#get_auc_lr_valid(train_df, y, C=1, seed=7, ratio = 0.8)

In [ ]:
model = word2vec.Word2Vec(train_df['list_w'], size=50, window=2, workers=4)
w2v = dict(zip(model.wv.index2word, model.wv.syn0))

In [ ]:
class mean_vectorizer(object):
    def __init__(self, word2vec):
        self.word2vec = word2vec
        self.dim = len(next(iter(w2v.values())))
    
    def fit(self, X):
        return self 

    def transform(self, X):
        return np.array([
            np.mean([self.word2vec[w] for w in words if w in self.word2vec] 
                    or [np.zeros(self.dim)], axis=0)
            for words in X
        ])
    

In [ ]:
class mean_vectorizer(object):
    def __init__(self, word2vec):
        self.word2vec = word2vec
        self.dim = len(next(iter(w2v.values())))
    
    def fit(self, X):
        return self 

    def transform(self, X):
        return np.array([
            np.mean([self.word2vec[w] for w in words if w in self.word2vec] 
                    or [np.zeros(self.dim)], axis=0)
            for words in X
        ])

# test

In [ ]:
class mean_vectorizer(object):
    def __init__(self, word2vec):
        self.word2vec = word2vec
        self.dim = len(next(iter(w2v.values())))
    
    def fit(self, X):
        return self 

    def transform(self, X):
        for words in X:
            
         return np.array([np.array([np.concatenate((self.word2vec[w] for w in words if w in self.word2vec\
                                                   else [np.zeros(self.dim)]), axis=0)]) for words in X])
      
    

In [ ]:
train_df_mean = mean_vectorizer(w2v).fit(train_df['list_w']).transform(train_df['list_w'])

In [ ]:
train_df_mean.shape

In [ ]:
19180350/50

In [ ]:
train_df_mean.reshape(253561,500)

In [ ]:
test_df['target'] = -1

In [ ]:
train_df = pd.concat([train_df,test_df],axis=0)

In [ ]:
train_df.shape

In [ ]:
model = word2vec.Word2Vec(train_df['list_w'], size=250, window=2, workers=4)
w2v = dict(zip(model.wv.index2word, model.wv.syn0))
#train_df_mean = mean_vectorizer(w2v).fit(train_df['list_w']).transform(train_df['list_w'])

In [ ]:
test_df = train_df[train_df['target']==-1]
test_df.drop(['target'],axis=1,inplace = True)

In [ ]:
test_df_mean = mean_vectorizer(w2v).fit(test_df['list_w']).transform(test_df['list_w'])

In [ ]:
train_df = train_df[train_df['target']!=-1]
train_df.drop(['target'],axis=1,inplace = True)

In [ ]:
train_df_mean = mean_vectorizer(w2v).fit(train_df['list_w']).transform(train_df['list_w'])

In [ ]:
test_df_mean = mean_vectorizer(w2v).fit(test_df['list_w']).transform(test_df['list_w'])

In [ ]:
np.savetxt("train_df_mean_250_3_otd.csv", train_df_mean, delimiter=",")

In [ ]:
np.savetxt("test_df_mean_250_3_otd.csv", test_df_mean, delimiter=",")

In [ ]:
y = np.array(train_df['target'])

In [ ]:
time_df = pd.read_csv('data/all_time_not_nan.csv')
time_df_t = time_df[time_df['target']==-1]
time_df_t.drop(['target'],axis=1,inplace = True)
time_df_t = StandardScaler().fit_transform(time_df_t)
time_df = time_df[time_df['target']!=-1]
time_df.drop(['target'],axis=1,inplace = True)
time_df = StandardScaler().fit_transform(time_df)

In [ ]:
train_df = np.concatenate((time_df, train_df_mean), axis=1)
test_df = np.concatenate((time_df_t, test_df_mean), axis=1)

In [ ]:
def split(train,y,ratio):
    idx = round(train.shape[0] * ratio)
    return train[:idx, :], train[idx:, :], y[:idx], y[idx:]
Xtr, Xval, ytr, yval = split(train_df, y,0.8)
Xtr.shape,Xval.shape,ytr.mean(),yval.mean()

In [ ]:
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Input
from keras.preprocessing.text import Tokenizer
from keras import regularizers

In [ ]:
model = Sequential()
model.add(Dense(128, input_dim=(Xtr.shape[1])))
model.add(Activation('relu'))
model.add(Dropout(0.5))
#model.add(Dense(128))
#model.add(Activation('relu'))
#model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

In [ ]:
from sklearn.metrics import roc_auc_score

In [ ]:
#kernel_regularizer=regularizers.l2(0.01),
#activity_regularizer=regularizers.l1(0.01)))

In [ ]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['binary_accuracy'])
#binary_accuracy,categorical_accuracy

In [ ]:
#from keras.callbacks import TensorBoard  
#tensorboard=TensorBoard(log_dir='./logs', write_graph=True, write_images=True)

history = model.fit(Xtr, ytr,
                    batch_size=128,
                    epochs=1,
                    validation_data=(Xval, yval),
                    #validation_split=0.1,
                    class_weight='auto',
                    verbose=2)

In [ ]:
classes = model.predict(Xval, batch_size=128)

In [ ]:
roc_auc_score(yval, classes)

In [ ]:
0.95647228127018291

In [ ]:
history = model.fit(train_df, y,
                    batch_size=128,
                    epochs=1,
                    #validation_data=(Xval, yval),
                    validation_split=0.01,
                    class_weight='auto',
                    verbose=2)

In [ ]:
classes = model.predict(test_df, batch_size=64)

In [ ]:
roc_auc_score(yval, classes)

In [ ]:
{0 : 1.,1: 50.}

In [ ]:
from keras.callbacks import EarlyStopping  
early_stopping=EarlyStopping(monitor='value_loss')  

history = model.fit(Xtr, ytr,
                    batch_size=64,
                    epochs=5,
                    validation_data=(Xval, yval),
                    verbose=1,
                    #validation_split=0.2,
                    callbacks=[early_stopping])

In [ ]:
# запишем его в файл, готовый для сабмита
#функция для записи прогнозов в файл
def write_to_submission_file(predicted_labels, out_file,
                             target='target', index_label="session_id"):
    predicted_df = pd.DataFrame(predicted_labels,
                                index = np.arange(1, predicted_labels.shape[0] + 1),
                                columns=[target])
    predicted_df.to_csv(out_file, index_label=index_label)
write_to_submission_file(classes, 'pred/nn_base_250_3_time_128_otherWord.csv')

In [ ]:
dtr = xgb.DMatrix(Xtr, label= ytr,missing = np.nan)
dval = xgb.DMatrix(Xval, label= yval,missing = np.nan)
watchlist = [(dtr, 'train'), (dval, 'eval')]
history = dict()

In [ ]:
# Fit a random forest to the training data, using 100 trees
from sklearn.ensemble import RandomForestClassifier
forest = RandomForestClassifier( n_estimators = 100 )

print "Fitting a random forest to labeled training data..."
forest = forest.fit( trainDataVecs, train["sentiment"] )

# Test & extract results 
result = forest.predict( testDataVecs )

# Write the test results 
output = pd.DataFrame( data={"id":test["id"], "sentiment":result} )
output.to_csv( "Word2Vec_AverageVectors.csv", index=False, quoting=3 )

In [ ]:
# создадим отдельный датафрейм, где будем работать со временем
time_df = pd.DataFrame(train_df[times]).reset_index()
test_df = pd.DataFrame(test_df[times]).reset_index()
#time_df['index'] = np.array(train_df['session_id'])
time_df['target'] = train_df['target']
# найдем время начала и окончания сессии
#time_df['min'] = train_df[times].min(axis=1)
#time_df['max'] = train_df[times].max(axis=1)

# вычислим длительность сессии и переведем в секунды
#time_df['seconds'] = (time_df['max'] - time_df['min']) / np.timedelta64(1, 's')

time_df.head()

In [ ]:
time_df.shape,test_df.shape

In [ ]:
test_df['target'] = -1

In [ ]:
time_df = pd.concat([time_df,test_df],axis=0)
time_df.shape

In [ ]:
time_df

In [ ]:
dates = pd.to_datetime(time_df['time1'])
time_df['Hour'] = dates.apply(lambda x: x.hour)
time_df['day'] = dates.apply(lambda x: x.day)
time_df['month'] = dates.apply(lambda x: x.month)
time_df['year'] = dates.apply(lambda x: x.year)
time_df['sec_in_week'+'time1'] = dates.apply(lambda x: 24*60*60*x.day+ 60*60*x.hour + 60*x.minute+x.second)
time_df['sec_in_day'+'time1'] = dates.apply(lambda x: 60*60*x.hour + 60*x.minute+x.second)
time_df['sec_in_hour'+"time1"] = dates.apply(lambda x: 60*x.minute+x.second)
time_df.drop(['time1'],axis=1,inplace = True)
for t in times[1:]:
    dates = pd.to_datetime(time_df[t])
    time_df['sec_in_week'+t] = dates.apply(lambda x: 24*60*60*x.day+ 60*60*x.hour + 60*x.minute+x.second)
    time_df['sec_in_day'+t] = dates.apply(lambda x: 60*60*x.hour + 60*x.minute+x.second)
    time_df['sec_in_hour'+t] = dates.apply(lambda x: 60*x.minute+x.second)
    time_df.drop([t],axis=1,inplace = True)
time_df.head()

In [ ]:
for i in range(len(times)-1):
    t1 = times[i]
    t2 = times[i+1]
    time_df['delta_sec'+t2] = (time_df['sec_in_hour'+t2]-time_df['sec_in_hour'+t1])%3600
time_df.head()

In [ ]:
time_df.shape

In [ ]:
times_delta = ['delta_sectime%s'% i for i in range(2,11)]
times_delta

In [ ]:
from scipy.stats import kurtosis,skew, mode

In [ ]:
time_df['max-min'] = time_df[times_delta].max(axis=1) - time_df[times_delta].min(axis=1)
time_df['std'] = time_df[times_delta].std(axis=1)
time_df['mean'] = time_df[times_delta].mean(axis=1)
time_df['median'] = time_df[times_delta].median(axis=1)
time_df['max'] = time_df[times_delta].max(axis=1)
time_df['min'] = time_df[times_delta].min(axis=1)
time_df['sem'] = time_df[times_delta].sem(axis=1)
time_df['sum'] = time_df[times_delta].sum(axis=1)
time_df['kurtosis'] = time_df[times_delta].kurtosis(axis=1)
time_df['skew'] = time_df[times_delta].skew(axis=1)
#time_df['mode'] = time_df[times_delta].mode(axis=1)#error
time_df['max+min'] = (time_df['max'] + time_df['min'])*0.5
time_df.head()

In [ ]:
for i in range(5,100,5):
    print(i)
    time_df['perc_{}'.format(i)] = time_df[times_delta].quantile(i/100,axis=1)
    

In [ ]:
def divide(a, b):
    if b>0:
        return a/b
    else:
        return -1

In [ ]:
time_df.reset_index(inplace = True)

In [ ]:
time_df

In [ ]:
time_df['mean-med'] = time_df['mean'] - time_df['median']
time_df['razmah_75_25'] = time_df['perc_75'] - time_df['perc_25']
time_df['koef_oscil'] = time_df.apply(lambda row: divide(row['max-min'], row['mean']), axis=1)
time_df['koef_oscil_med'] = time_df.apply(lambda row: divide(row['max-min'], row['median']), axis=1)
time_df['koef_var_'] = time_df.apply(lambda row: divide(row['std'], row['mean']), axis=1)
time_df['koef_var_med'] = time_df.apply(lambda row: divide(row['std'], row['median']), axis=1)
time_df['koef_quant_'] = time_df.apply(lambda row: divide(row['razmah_75_25'], row['mean']), axis=1)
time_df['koef_quant_med'] = time_df.apply(lambda row: divide(row['razmah_75_25'], row['median']), axis=1)
time_df['mean/med'] = time_df.apply(lambda row: divide(row['mean'], row['median']), axis=1)


In [ ]:
for i in range(10,100,5):
    print(i)
    time_df['dperc_{}'.format(i)] = time_df['perc_{}'.format(i)]-time_df['perc_{}'.format(i-5)]

In [ ]:
time_df.drop(['level_0','index'],axis=1,inplace = True)

In [ ]:
time_df.to_csv('data/all_time_nan.csv',index = False)

In [ ]:
time_df = time_df.fillna(-9999)

In [ ]:
time_df.to_csv('data/all_time_not_nan.csv',index = False)

In [ ]:
time_df.shape

In [ ]:
time_df = pd.read_csv('data/all_time_not_nan.csv')
time_df_t = time_df[time_df['target']==-1]
time_df_t.drop(['target'],axis=1,inplace = True)
time_df_t = StandardScaler().fit_transform(time_df_t)
time_df = time_df[time_df['target']!=-1]
time_df.drop(['target'],axis=1,inplace = True)
time_df = StandardScaler().fit_transform(time_df)

In [ ]:
time_df.shape,train_df_mean.shape,test_df_mean.shape,time_df_t.shape,y.shape

In [ ]:
train_df = np.concatenate((time_df, train_df_mean), axis=1)
test_df = np.concatenate((time_df_t, test_df_mean), axis=1)

In [ ]:
train_df.shape

In [ ]:
def split(train,y,ratio):
    idx = round(train.shape[0] * ratio)
    return train[:idx], train[idx:], y[:idx], y[idx:]
Xtr, Xval, ytr, yval = split(time_df, y,0.8)
Xtr.shape,Xval.shape,ytr.mean(),yval.mean()

In [ ]:
Xtr, Xval, ytr, yval = train_test_split(time_df[feat], time_df.target,random_state = 7, test_size=0.25)

In [ ]:
def split(train,y,ratio):
    idx = round(train.shape[0] * ratio)
    return train[:idx, :], train[idx:, :], y[:idx], y[idx:]
Xtr, Xval, ytr, yval = split(train_df, y,0.8)
Xtr.shape,Xval.shape,ytr.mean(),yval.mean()

In [ ]:
import xgboost as xgb

In [ ]:
dtr = xgb.DMatrix(Xtr, label= ytr,missing = np.nan)
dval = xgb.DMatrix(Xval, label= yval,missing = np.nan)
watchlist = [(dtr, 'train'), (dval, 'eval')]
history = dict()

In [ ]:
params = {
    'max_depth': 6,
    'eta': 0.025,
    'nthread': 4,
    #'gamma' : 1,
    'alpha' : 1,
    'subsample': 0.85,
    'eval_metric': ['auc'],
    'objective': 'binary:logistic',
    #'objective': 'reg:linear',
    'colsample_bytree': 0.9,
    'min_child_weight': 20,
    'scale_pos_weight':(1)/y.mean(),
    'seed':7
}

In [ ]:
model_new = xgb.train(params, dtr, num_boost_round=1900, evals=watchlist,evals_result=history, verbose_eval=20)

In [ ]:
[80]	train-auc:0.961231	eval-auc:0.907617 250 word2vec
        train-auc:0.972142	eval-auc:0.915109 1/mean

In [ ]:
[80]	train-auc:0.992534	eval-auc:0.96465

In [ ]:
print(get_auc_lr_valid(time_df[feat], time_df.target))

In [ ]:
# создадим отдельный датафрейм, где будем работать со временем
time_df = pd.DataFrame(index=train_df.index)
time_df['target'] = train_df['target']

# найдем время начала и окончания сессии
time_df['min'] = train_df[times].min(axis=1)
time_df['max'] = train_df[times].max(axis=1)

# вычислим длительность сессии и переведем в секунды
time_df['seconds'] = (time_df['max'] - time_df['min']) / np.timedelta64(1, 's')

time_df.head()

In [ ]:
60*60

In [ ]:
# наша целевая переменная
y_train = train_df['target']

# объединенная таблица исходных данных
full_df = pd.concat([train_df.drop('target', axis=1), test_df])

# индекс, по которому будем отделять обучающую выборку от тестовой
idx_split = train_df.shape[0]

In [ ]:
# табличка с индексами посещенных сайтов в сессии
full_sites = full_df[sites]
full_sites.head()

In [ ]:
# последовательность с индексами
sites_flatten = full_sites.values.flatten()

# искомая матрица
full_sites_sparse = csr_matrix(([1] * sites_flatten.shape[0],
                                sites_flatten,
                                range(0, sites_flatten.shape[0]  + 10, 10)))[:, 1:]

In [ ]:
# Сколько места занимает разреженная матрица в памяти?
print('{0} elements * {1} bytes = {2} bytes'.format(full_sites_sparse.count_nonzero(), 8, 
                                                    full_sites_sparse.count_nonzero() * 8))
# или сразу вот так:
print('sparse_matrix_size = {0} bytes'.format(full_sites_sparse.data.nbytes))

In [ ]:
sites_dict.head()

In [ ]:
x = 'docs.jboss.org'
re.findall(".\w+",x)[0]

In [ ]:
def resl2(x):
    return re.findall("\w+.",x)[-1]
    #return x[-2:]
def resl3(x):
    return re.findall(".\w+",x)[0]
def resl4(x):
    return x[-4:]

In [ ]:
sites_dict['l2'] = sites_dict['site'].apply(resl2)
sites_dict['l3'] = sites_dict['site'].apply(resl3)
sites_dict['l4'] = sites_dict['site'].apply(resl4)

In [ ]:
sites_dict['l3'].value_counts()

In [ ]:
sites_dict['target'] = train_df['target']

In [ ]:
sites_dict.groupby('l2')['target'].count().plot(kind='bar', rot=45)
sites_dict.groupby('l2')['target'].sum().plot(kind='bar',colors = 'r', rot=45)
plt.show()

In [ ]:
train_df_mean = pd.DataFrame()

In [ ]:
full_tr = xgb.DMatrix(train_df, label= y,missing = np.nan)
watchlist = [(full_tr, 'train')]
history = dict()

In [ ]:
model_full = xgb.train(params, full_tr, num_boost_round=100, evals=watchlist,evals_result=history, verbose_eval=20)

In [ ]:
test_df_mean.shape,train_df_mean.shape

In [ ]:
#функция для записи прогнозов в файл
def write_to_submission_file(predicted_labels, out_file,
                             target='target', index_label="session_id"):
    predicted_df = pd.DataFrame(predicted_labels,
                                index = np.arange(1, predicted_labels.shape[0] + 1),
                                columns=[target])
    predicted_df.to_csv(out_file, index_label=index_label)

In [ ]:
full_test = xgb.DMatrix(test_df, missing = np.nan)
y_test = model_full.predict(full_test)



In [ ]:
# запишем его в файл, готовый для сабмита
#функция для записи прогнозов в файл
def write_to_submission_file(predicted_labels, out_file,
                             target='target', index_label="session_id"):
    predicted_df = pd.DataFrame(predicted_labels,
                                index = np.arange(1, predicted_labels.shape[0] + 1),
                                columns=[target])
    predicted_df.to_csv(out_file, index_label=index_label)
write_to_submission_file(y_test, 'pred/xgb_base_250_3_time_100.csv')